<a href="https://colab.research.google.com/github/EdenErlich/NLP_project/blob/main/Copy_of_NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl">

בשלב זה בנינו תהליך לאיסוף, ניקוי ושמירת הנתונים בצורה נוחה לשימוש חוזר.  
תחילה העלינו ל-Colab קובץ ZIP עם דפי תוכן מאתר JCT, חילצנו אותו וקראנו את קובץ ה-JSON שבתוכו כדי לאסוף את כל כתובות ה-URL הייחודיות.  
לאחר מכן ביצענו סריקה (scraping) של כל אחד מהדפים: הורדנו את התוכן שלהם, ניקינו אותו מקוד HTML, סקריפטים ועיצובים והשארנו טקסט נקי בלבד.  
את כל המידע — כתובת ותוכן נקי — שמרנו בקובץ חדש בשם `lev_scraped_with_content.json`, כך שבפעמים הבאות נוכל לטעון את הקובץ הזה ישירות ולא נצטרך לבצע שוב את הסריקה.  
בכך הכנו לעצמנו בסיס נתונים נקי ומוכן לאימון או אינדוקס.

</div>

In [ ]:
!pip install sentence-transformers faiss-cpu transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 68.5 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu


In [ ]:
import json, re
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# טען את הדאטה
#encoding='utf-8' נועד לוודא שהתווים בעברית או תווים מיוחדים לא ייפגעו.


with open('/content/drive/MyDrive/jct_scraped_pages.json', encoding='utf-8') as f:
    data = json.load(f)
#מוסיפה פה דרך להמיר נכון את הקובץ לDF
import json
import pandas as pd



# המר למבנה נכון לדאטהפריים
rows = []
for url, info in data.items():
    rows.append({
        "url": url,
        "title": info.get("title", ""),
        "content": info.get("content", "")
    })

df = pd.DataFrame(rows)

#לבדוק שבאמת יש טבלה אחרת אין מה להמשיך יההי שגאות תיקון לקוד
df.head()

,url,title,content
0,https://www.jct.ac.il/en,Home Page - Jerusalem College of Technology,Home Page - Jerusalem College of Technology דל...
1,https://www.jct.ac.il/en/,Home Page - Jerusalem College of Technology,Home Page - Jerusalem College of Technology דל...
2,https://www.jct.ac.il/en/campuses/campus-lev/c...,Contact Information - Jerusalem College of Tec...,Contact Information - Jerusalem College of Tec...
3,https://www.jct.ac.il/en/about-jct/,About JCT - Jerusalem College of Technology,About JCT - Jerusalem College of Technology דל...
4,https://www.jct.ac.il/en/jct-admins/,JCT Admins - Jerusalem College of Technology,JCT Admins - Jerusalem College of Technology ד...


In [ ]:
print(df.columns)


Index(['url', 'title', 'content'], dtype='object')


In [ ]:
# נקה טקסט
#בשביל הכנה ל EMBEDING
def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
#ישום הניקוי
df['clean_content'] = df['content'].apply(clean_text)
# שיפור ניקוי:
df['clean_content'] = df['clean_content'].str.replace(r'http\S+', '', regex=True)
df['clean_content'] = df['clean_content'].str.replace(r'[^\w\sא-ת]', '', regex=True)

# סינון טקסטים קצרים או ריקים:
df = df[~df['clean_content'].str.strip().eq('')]
df['text_length'] = df['clean_content'].apply(len)
df = df[df['text_length'] > 100]


# כמה שורות?
print(f"✅ מספר שורות בדאטה: {len(df)}")

# אורך טקסטים
# לסינון טקסטים קצרים מדי/ארוכים מדי, ולניתוח.
df['text_length'] = df['clean_content'].apply(len)
print("📊 סטטיסטיקות אורך הטקסטים:")
print(df['text_length'].describe())

# דוגמאות
#מאפשר לראות האם הניקוי עבד, והאם יש טקסטים בעייתיים.


for idx, row in df.sample(3).iterrows():
    print(f"\n--- טקסט לדוגמה ({row['text_length']} תווים) ---")
    print(row['clean_content'][:1000], "...")

# אחוז פסקאות (newlines)
#שימושי לדעת אם הטקסטים מכילים מבנה של פסקאות או שהם שורה אחת ארוכה".
has_newlines = df['clean_content'].apply(lambda x: '\n' in x).mean()
print(f"\n📄 אחוז טקסטים עם פסקאות (\\n): {has_newlines*100:.1f}%")

#נותן אינדיקציה כמה מהטקסטים רלוונטיים לנושא מסוים.
keywords = df['clean_content'].str.contains(r'בגרות|85|תנאי קבלה', case=False, na=False).mean()
print(f"📄 אחוז טקסטים עם מילות מפתח רלוונטיות: {keywords*100:.1f}%")


✅ מספר שורות בדאטה: 383
📊 סטטיסטיקות אורך הטקסטים:
count      383.000000
mean      7036.986945
std       6883.271468
min       2751.000000
25%       3925.000000
50%       4876.000000
75%       6896.500000
max      43178.000000
Name: text_length, dtype: float64

--- טקסט לדוגמה (7980 תווים) ---
BOR HATORAH 2  Jerusalem College of Technology דלג לתוכן This website uses cookies to ensure you get the best experience on our website privacy policy and terms of use Map Contact Us Virtual Tour Fr ע About About JCT JCT Admins Community Engagement Financial Statements Registration guide Campuses Campus Lev Campus Tal Campus Lustig Special Programs Programs in English Mechina Haredi Program Ethiopian Program Atudai Program Publications Friends Tal Campus Project Academic Calendar High Contrast Ways to Give Click here to apply  Common Questions What are the admission requirements High School Diploma SATACTTIL Israeli Exam Personal Interview PrecaluclusCalculus Background for Computer Science only 

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:

def smart_chunk(text, max_chars=384):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sent in sentences:
        if len(current_chunk) + len(sent) <= max_chars:
            current_chunk += " " + sent
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sent
    if len(current_chunk.strip()) > 50:
     chunks.append(current_chunk.strip())


    return chunks

chunks = []
for idx, row in df.iterrows():
    for i, chunk in enumerate(smart_chunk(row['clean_content'])):
        chunks.append({
            'id': f"{idx}_{i}",
            'text': chunk,
            #מוסיפה פה שידרוג
            'doc_id': idx

        })

chunks_df = pd.DataFrame(chunks)
print(f"✅ נוצרו {len(chunks_df)} מקטעים.")
chunks_df.head()


✅ נוצרו 766 מקטעים.


,id,text,doc_id
0,0_0,,0
1,0_1,Home Page Jerusalem College of Technology דלג...,0
2,1_0,,1
3,1_1,Home Page Jerusalem College of Technology דלג...,1
4,2_0,,2


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
#מודל שמתאים גם לעברית
embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device='cuda')

embeddings = []
batch_size = 16
texts = chunks_df['text'].tolist()

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    emb = embed_model.encode(batch, show_progress_bar=True)
    embeddings.append(emb)

embeddings = np.vstack(embeddings).astype("float32")
print(f"✅ סיימנו, embeddings shape: {embeddings.shape}")

#  לוודא שאין אפסים
print(np.isnan(embeddings).sum())
print(np.linalg.norm(embeddings, axis=1).min())


faiss.normalize_L2(embeddings)

index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

print(f"✅ אינדקס מוכן עם {len(embeddings)} מקטעים.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ סיימנו, embeddings shape: (766, 384)
0
2.8234591
✅ אינדקס מוכן עם 766 מקטעים.


In [ ]:
print(chunks_df['text'].value_counts().head(10))


text
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
# 🔷 שמור embeddings
np.save('/content/drive/MyDrive/embeddings.npy', embeddings)
print("✅ embeddings נשמרו בדרייב")

# 🔷 שמור FAISS index
faiss.write_index(index, '/content/drive/MyDrive/faiss_index.index')
print("✅ אינדקס FAISS נשמר בדרייב")

# 🔷 שמור את chunks_df כ־CSV
chunks_df.to_csv('/content/drive/MyDrive/chunks_df.csv', index=False)
print("✅ chunks_df נשמר בדרייב")


✅ embeddings נשמרו בדרייב
✅ אינדקס FAISS נשמר בדרייב
✅ chunks_df נשמר בדרייב


In [ ]:
def retrieve(query, k=5):
    query_emb = embed_model.encode([query]).astype("float32")
    faiss.normalize_L2(query_emb)
    D, I = index.search(query_emb, k)
    return I[0], D[0]

question = "What are the admission requirements?"
true_keywords = ["High School Diploma", "SAT", "Personal Interview"]

ids, scores = retrieve(question)

print("\n🔷 פסקאות שנשלפו:")
for idx, score in zip(ids, scores):
    text = chunks_df.iloc[idx]['text']
    print(f"\n--- (score: {score:.3f}) ---\n{text}")
    if any(kw.lower() in text.lower() for kw in true_keywords):
        print("✅ פסקה רלוונטית נמצאה כאן!")



🔷 פסקאות שנשלפו:

--- (score: 0.326) ---
120 Haredi Female Students Receive Degrees from JCT  Jerusalem College of Technology דלג לתוכן This website uses cookies to ensure you get the best experience on our website privacy policy and terms of use Map Contact Us Virtual Tour Fr ע About About JCT JCT Admins Community Engagement Financial Statements Registration guide Campuses Campus Lev Campus Tal Campus Lustig Special Programs Programs in English Mechina Haredi Program Ethiopian Program Atudai Program Publications Friends Tal Campus Project Academic Calendar High Contrast Ways to Give Click here to apply  Common Questions What are the admission requirements High School Diploma SATACTTIL Israeli Exam Personal Interview PrecaluclusCalculus Background for Computer Science only How much is tuition Approximately 3800 per year At exchange rate of 355 NIS per dollar and current public tuition rates set by the Council for Higher Education This fee does not include room board and Beit Midrash i

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
import pandas as pd

# 🔷 שאלות ותשובות נכונות שאני מצפה
questions = [
    {
        "question": "What are the admission requirements?",
        "expected_keywords": ["High School Diploma", "SAT", "Personal Interview"]
    },
    {
        "question": "How much is tuition?",
        "expected_keywords": ["$3,800", "tuition"]
    },
    {
        "question": "When does the semester start?",
        "expected_keywords": ["September", "school year", "first semester"]
    },
]

results = []

# 🔷 הפונקציה שלנו לאחזור
def retrieve(query, k=5):
    query_clean = re.sub(r'[^\w\s]', '', query.lower())

    query_emb = embed_model.encode([query_clean]).astype("float32")
    faiss.normalize_L2(query_emb)
    D, I = index.search(query_emb, k)
    seen_texts = set()
    unique_indices = []
    unique_scores = []

    for idx, score in zip(I[0], D[0]):
        text = chunks_df.iloc[idx]['text']
        if text not in seen_texts:
            seen_texts.add(text)
            unique_indices.append(idx)
            unique_scores.append(score)
        if len(unique_indices) == 5:
            break

    return unique_indices, unique_scores


# 🔷 עבור כל שאלה — נבדוק אם המידע הופיע
for q in questions:
    ids, scores = retrieve(q['question'], k=10)

    found = False
    retrieved_texts = []

    for idx, score in zip(ids, scores):
        text = chunks_df.iloc[idx]['text']
        retrieved_texts.append(text)
        if any(kw.lower() in text.lower() for kw in q['expected_keywords']):
            found = True

    results.append({
        "Question": q['question'],
        "Found?": "✅" if found else "❌",
        "Top Match": retrieved_texts[0][:150] + "...",
    })

# 🔷 טבלה יפה
df_results = pd.DataFrame(results)
print(df_results)

# 🔷 דיוק כולל
accuracy = (df_results['Found?'] == "✅").mean()
print(f"\n🎯 Recall@5: {accuracy*100:.1f}%")


                               Question Found?  \
0  What are the admission requirements?      ✅   
1                  How much is tuition?      ✅   
2         When does the semester start?      ✅   

                                           Top Match  
0  120 Haredi Female Students Receive Degrees fro...  
1  Cross River Donates 100000 for Scholarship Fun...  
2  JCT opens 2019 school year on campus  Jerusale...  

🎯 Recall@5: 100.0%


In [ ]:
!pip install transformers accelerate bitsandbytes


In [ ]:
from huggingface_hub import login
login()  # ← תדביקו את הטוקן שלכם כאן כקלט


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from transformers import BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # or "fp4"
    bnb_4bit_compute_dtype=torch.bfloat16,  # or torch.float16
    # bnb_4bit_use_double_quant=False, # optional
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    # torch_dtype="auto" # This might cause issues with 4-bit quantization
    quantization_config=bnb_config
)

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def make_short_context(texts, max_chars=3500):
    context = ""
    for t in texts:
        if len(context) + len(t) > max_chars:
            break
        context += t.strip() + "\n"
    return context


In [ ]:
def generate_answer(question, context, max_new_tokens=150):
    prompt = f"[INST] Context:\n{context}\n\nQuestion: {question}\nAnswer: [/INST]"
    output = qa_pipeline(prompt, max_new_tokens=max_new_tokens, do_sample=False)[0]['generated_text']


    # חיתוך לפי תגיות INSTRUCT
    if "[/INST]" in output:
        return output.split("[/INST]")[-1].strip()
    else:
        return output.strip()


In [ ]:
results = []

for idx, q in enumerate(questions):
    print(f"Processing question {idx+1}/{len(questions)}: {q['question'][:30]}...")  # הדפס התקדמות

    question = q["question"]
    expected_keywords = q["expected_keywords"]

    # אחזור פסקאות רלוונטיות
    ids, _ = retrieve(question)
    top_chunks = [chunks_df.iloc[i]['text'] for i in ids]
    context = make_short_context(top_chunks)

    # הפקת תשובה
    answer = generate_answer(question, context)

    # בדיקת מילות מפתח
    found_keywords = [kw for kw in expected_keywords if kw.lower() in answer.lower()]
    found = "✅" if found_keywords else "❌"

    results.append({
        "Question": question,
        "Answer": answer,
        "Matched Keywords": ", ".join(found_keywords),
        "Success": found
    })


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing question 1/3: What are the admission require...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing question 2/3: How much is tuition?...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing question 3/3: When does the semester start?...


In [ ]:
import pandas as pd

df_results = pd.DataFrame(results)
pd.set_option('display.max_colwidth', 200)
display(df_results)

accuracy = (df_results['Success'] == "✅").mean()
print(f"\n🎯 דיוק כולל (Answer Accuracy): {accuracy*100:.1f}%")

,Question,Answer,Matched Keywords,Success
0,What are the admission requirements?,"To provide an accurate answer, I would need to know the specific program or institution you are referring to. Generally, admission requirements can include various factors such as:\n\n* Academic q...","High School Diploma, SAT",✅
1,How much is tuition?,"To provide an accurate answer, I would need to know which specific educational institution and program you are inquiring about. Tuition fees vary greatly depending on the level of education (under...",tuition,✅
2,When does the semester start?,"To provide an accurate answer, I would need to know which semester and which educational institution you are referring to. Semesters can start at different times depending on the institution. For ...",September,✅



🎯 דיוק כולל (Answer Accuracy): 100.0%


In [ ]:
# שאלה לדוגמה
question = "What are the admission requirements?"

# נשלוף את הקונטקסט מהאחזור הקודם
ids, scores = retrieve(question)
context = "\n".join(chunks_df.iloc[i]['text'] for i in ids)

# יוצרים תשובה
generated = generate_answer(question, context)

print("🔷 תשובה שנוצרה:")
print(generated)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔷 תשובה שנוצרה:
The admission requirements for Jerusalem College of Technology (JCT) include a High School Diploma, SAT or TIL exam results, a personal interview, and a background in Precalculus and Calculus for Computer Science students only. The tuition fee is approximately 3800 shekels per year, but it does not include room and board or Beit Midrash fees if required. The school year starts after Sukkot each year, and the college provides breaks for Jewish holidays. To register, please complete an online registration form and pay a nonrefundable registration fee of 300 shekels. The program length varies for Computer Science and Business Administration degrees, with Computer Science having more full


In [ ]:
!pip install -U bitsandbytes